In [ ]:
# if running in Colab run this first, otherwise make sure flair[word-embeddings] is installed
import torch
!pip install flair[word-embeddings]
import flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from pathlib import Path

from flair.data import Corpus, Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, CharacterEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer


if __name__ == "__main__":
    # load corpus
    columns = {0: 'text', 1: 'ner'}
    data_folder = '/home/dataset'
    corpus: Corpus = ColumnCorpus(data_folder, columns,
                                train_file='train.txt',
                                test_file='test.txt',
                                dev_file='dev.txt',
                                column_delimiter='\t')

    # extract the labels from the corpus
    label_type = 'ner'
    label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)

    # train model
    embeddings = FlairEmbeddings('news-forward-fast')

    embedding_types = [
        WordEmbeddings('glove'),
        FlairEmbeddings('news-forward'),
        FlairEmbeddings('news-backward')
    ]

    embeddings = StackedEmbeddings(embeddings=embedding_types)

    tagger = SequenceTagger(hidden_size=256,
                            embeddings=embeddings,
                            tag_dictionary=label_dict,
                            tag_type=label_type,
                            tag_format="BIO")

    trainer = ModelTrainer(tagger, corpus)

    trainer.train('/home/model',
                  learning_rate=0.05,
                  mini_batch_size=32,
                  max_epochs=25)


2025-04-29 21:28:25,777 Reading data from /home/dataset
2025-04-29 21:28:25,779 Train: /home/dataset/train.txt
2025-04-29 21:28:25,779 Dev: /home/dataset/dev.txt
2025-04-29 21:28:25,780 Test: /home/dataset/test.txt
2025-04-29 21:28:27,550 Computing label dictionary. Progress:


4it [00:00, 6428.05it/s]
6338it [00:00, 44566.02it/s]

2025-04-29 21:28:27,702 Dictionary created for label 'ner' with 3 values: PER (seen 3101 times), ORG (seen 2267 times), LOC (seen 1996 times)


2025-04-29 21:28:32,464 SequenceTagger predicts: Dictionary with 7 tags: O, B-PER, I-PER, B-ORG, I-ORG, B-LOC, I-LOC
2025-04-29 21:28:32,750 ----------------------------------------------------------------------------------------------------
2025-04-29 21:28:32,751 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196,

/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:107: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


...

100%|██████████| 16/16 [00:04<00:00,  3.86it/s]

2025-04-29 21:39:44,351 DEV : loss 0.10540113598108292 - f1-score (micro avg)  0.6932


2025-04-29 21:39:44,425  - 1 epochs without improvement
2025-04-29 21:39:46,138 ----------------------------------------------------------------------------------------------------
2025-04-29 21:39:46,140 Loading model from best epoch ...
2025-04-29 21:39:47,614 SequenceTagger predicts: Dictionary with 9 tags: O, B-PER, I-PER, B-ORG, I-ORG, B-LOC, I-LOC, <START>, <STOP>


100%|██████████| 32/32 [00:12<00:00,  2.64it/s]


2025-04-29 21:40:00,140 
Results:
- F-score (micro) 0.7216
- F-score (macro) 0.6719
- Accuracy 0.6063

By class:
              precision    recall  f1-score   support

         PER     0.8304    0.8315    0.8309      1602
         ORG     0.5786    0.4785    0.5238       792
         LOC     0.7630    0.5831    0.6610       602

   micro avg     0.7584    0.6883    0.7216      2996
   macro avg     0.7240    0.6310    0.6719      2996
weighted avg     0.7503    0.6883    0.7156      2996

2025-04-29 21:40:00,140 ----------------------------------------------------------------------------------------------------
